In [5]:
import numpy as np
from Optimization.RiemannianBCD import RiemannianBlockCoordinateDescent
from Optimization.RiemannianGAS import RiemannianGradientAscentSinkhorn
import torch
import matplotlib.pyplot as plt
import time

In [3]:
def T(x,d,dim=2):
    assert dim <= d
    assert dim >= 1
    assert dim == int(dim)
    return x + 2*np.sign(x)*np.array(dim*[1]+(d-dim)*[0])

def fragmented_hypercube(n,d,dim):
    assert dim <= d
    assert dim >= 1
    assert dim == int(dim)
    
    a = (1./n) * np.ones(n)
    b = (1./n) * np.ones(n)
    
    # First measure : uniform on the hypercube
    X = np.random.uniform(-1, 1, size=(n,d))

    # Second measure : fragmentation
    Y = T(np.random.uniform(-1, 1, size=(n,d)), d, dim)
    
    return a,b,X,Y

In [4]:
def InitialStiefel(d, k):
    U = np.random.randn(d, k)
    q, r = np.linalg.qr(U)
    return q

In [9]:
# X,Y in R^{n x d}
# theta in R^{d x 1} (just borrowing existing conventions for higher k)
def sliced_objective(X,Y,theta,p=2.0):
    theta = theta.flatten()
    assert X.shape == Y.shape
    assert X.shape[1] == theta.shape[0]
    Xtheta = X.dot(theta)
    Ytheta = Y.dot(theta)
    return np.mean(np.abs(np.sort(Xtheta) - np.sort(Ytheta))**p)**(1/p)

#repeated
def proj_wp(X, Y, theta, p=2):
    N, d = X.shape
    theta = theta.flatten()
    xproj = np.matmul(X, theta)
    yproj = np.matmul(Y, theta)
    return np.mean(np.abs((np.sort(xproj) - np.sort(yproj)))**p)**(1/p)

def norm(x):
    return np.sqrt(sum(x**2))

def samp_sph(d):
    x = np.random.normal(size = d)
    return x/norm(x)

In [39]:
def subg_step(X, Y, theta, alpha):
    N, d = X.shape
    theta_X = np.matmul(X, theta)
    theta_Y = np.matmul(Y, theta)

    X_ind = np.argsort(theta_X)
    Y_ind = np.argsort(theta_Y)
    grad = 2*np.dot(theta_X[X_ind] - theta_Y[Y_ind], X[X_ind,:] - Y[Y_ind,:])/N
    newtheta = (theta + alpha*grad)
    return newtheta/norm(newtheta)
             
def msw2_distance_subg(X, Y, n_step, theta0):
    N, d = X.shape
    alpha = np.ones(n_step) # constant step size, alternative:/np.sqrt(range(1,n_step + 1))
    theta = theta0
    wp_dist = np.zeros(n_step)

    time_iter = np.zeros(n_step+1)
    U_iter = np.zeros((n_step+1,theta.shape[0],1))
    U_iter[0,:,:] = theta0[np.newaxis].T

    for i in range(n_step):
        tic = time.perf_counter()

        theta = subg_step(X, Y, theta, alpha[i])
        wp_dist[i] = proj_wp(X, Y, theta)

        toc = time.perf_counter()
        time_iter[i + 1] = time_iter[i] + toc - tic
        U_iter[i+1,:,:] = theta[np.newaxis].T
    return (U_iter, time_iter)

In [46]:
def get_objective_iter(X,Y, U_iter):
    return np.array([sliced_objective(X,Y,U_iter[i,:,:]) for i in range(U_iter.shape[0])])

In [24]:
d = 100 # Dimension
n = 500 # Number of samples
k = 1  # Subspace dimension
dim = 10 # fragmentation dimension
a,b,X,Y = fragmented_hypercube(n,d,dim)
rotation = InitialStiefel(d,d)
X = X.dot(rotation)
Y = Y.dot(rotation)
    
U0 = InitialStiefel(d, k)

gamma = 0.001
eta = 0.2

In [ ]:
# Run max-sliced W2 computations using three algorithms with varied dimension

logs = {}
max_iter = 200
K = 10

for d in [10, 20, 50, 100]:
    print(f'd:{d}')

    n = 500 # Number of samples
    k = 1  # Subspace dimension
    dim = 10 # fragmentation dimension
    a,b,X,Y = fragmented_hypercube(n,d,dim)
    rotation = InitialStiefel(d,d)
    X = X.dot(rotation)
    Y = Y.dot(rotation)
        
    U0 = InitialStiefel(d, k)

    gamma = 0.001
    eta = 0.2

    log = []

    for i in range(K):
        print(i)
        params = {'eta':eta, 'tau':gamma/eta, 'max_iter':max_iter, 'threshold':0.1, 'verbose':True}
        algo1 = RiemannianGradientAscentSinkhorn(**params)
        (U_iter_RAGAS, time_iter_RAGAS) = algo1.run_RAGAS(a, b, X, Y, k, U0)
        objective_iter_RAGAS = get_objective_iter(X,Y,U_iter_RAGAS)

        params = {'eta':eta, 'tau':gamma, 'max_iter':max_iter, 'threshold':0.1, 'verbose':True}
        algo2 = RiemannianBlockCoordinateDescent(**params)
        (U_iter_RABCD, time_iter_RABCD) = algo2.run_RABCD(a, b, X, Y, k, U0)
        objective_iter_RABCD = get_objective_iter(X,Y,U_iter_RABCD)

        (U_iter_subg, time_iter_subg) = msw2_distance_subg(X, Y, max_iter, U0.flatten())
        objective_iter_subg = get_objective_iter(X,Y,U_iter_subg)

        log.append(((U_iter_RAGAS, objective_iter_RAGAS, time_iter_RAGAS), 
                    (U_iter_RABCD, objective_iter_RABCD, time_iter_RABCD), 
                    (U_iter_subg, objective_iter_subg, time_iter_subg)))

    logs[d] = log

In [ ]:
# Computation Time Plots

colors = {}
colors[20] = 'g'
colors[50] = 'c'
colors[100] = 'tab:orange'
avgs = {}
for d in [20,50,100]:
    log = logs[d]
    time_iter_RAGAS_avg = np.zeros(log[0][0][2].shape)
    time_iter_RABCD_avg = np.zeros(log[0][0][2].shape)
    time_iter_subg_avg = np.zeros(log[0][0][2].shape)
    for k in range(K):
        time_iter_RAGAS_avg += log[k][0][2]
        time_iter_RABCD_avg += log[k][1][2]
        time_iter_subg_avg += log[k][2][2]
    time_iter_RAGAS_avg /= K
    time_iter_RABCD_avg /= K
    time_iter_subg_avg /= K
    avgs[d] = (time_iter_RAGAS_avg, time_iter_RABCD_avg, time_iter_subg_avg)

for d in [20,50,100]:
    plt.plot(avgs[d][2], '-', label=f'subgradient ascent, d={d}', color=colors[d])
for d in [20,50,100]:
    plt.plot(avgs[d][1], '--', label=f'RABCD, d={d}', color=colors[d])
for d in [20,50,100]:
    plt.plot(avgs[d][0], '-.', label=f'RAGAS, d={d}', color=colors[d])

plt.legend()
plt.ylabel('time elapsed (seconds)')
plt.xlabel('# ascent steps')
plt.title('Computation Time for $\overline{W}_2$ Algorithms')

In [ ]:
# Iteration Complexity Plots

colors = {}
colors[20] = 'g'
colors[50] = 'c'
colors[100] = 'tab:orange'
avgs = {}
for d in [20,50,100]:
    log = logs[d]
    time_iter_RAGAS_avg = np.zeros(log[0][0][2].shape)
    time_iter_RABCD_avg = np.zeros(log[0][0][2].shape)
    time_iter_subg_avg = np.zeros(log[0][0][2].shape)
    for k in range(K):
        time_iter_RAGAS_avg += log[k][0][2]
        time_iter_RABCD_avg += log[k][1][2]
        time_iter_subg_avg += log[k][2][2]
    time_iter_RAGAS_avg /= K
    time_iter_RABCD_avg /= K
    time_iter_subg_avg /= K
    avgs[d] = (time_iter_RAGAS_avg, time_iter_RABCD_avg, time_iter_subg_avg)

for d in [20,50,100]:
    plt.plot(logs[d][0][2][1], '-', label=f'subgradient ascent, d={d}', color=colors[d])
for d in [20,50,100]:
    plt.plot(logs[d][0][1][1], '--', label=f'RABCD, d={d}', color=colors[d])
for d in [20,50,100]:
    plt.plot(logs[d][0][0][1], '-.', label=f'RAGAS, d={d}', color=colors[d])

plt.legend()
plt.ylabel('$W_2$ along current slice')
plt.xlabel('# ascent steps')
plt.title('Iteration Complexity for $\overline{W}_2$ Algorithms')
plt.ylim(1,2.3);